In [0]:
# Lê a tabela silver já limpa (sem duplicados)
df = spark.table("saas_project.core.bronze_raw")


# Exibe os dados no notebook
display(df.limit(10))

id,nome,idade,email,salario,cidade,ingestion_time
1,Pessoa 0,26,user0@exemplo.com,2036.32,null,2026/02/18-04:02:12
2,Pessoa 1,68,user1@exemplo.com,3705.09,Rio,2026/02/18-04:02:12
3,Pessoa 2,67,user2@exemplo.com,5693.16,São Paulo,2026/02/18-04:02:12
4,Pessoa 3,38,user3@exemplo.com,9376.51,Belo Horizonte,2026/02/18-04:02:12
5,Pessoa 4,53,user4@exemplo.com,6392.52,null,2026/02/18-04:02:12
6,Pessoa 5,49,user5@exemplo.com,7172.05,Rio,2026/02/18-04:02:12
7,Pessoa 6,null,user6@exemplo.com,3802.53,null,2026/02/18-04:02:12
8,Pessoa 7,47,user7@exemplo.com,4748.2,null,2026/02/18-04:02:12
9,Pessoa 8,27,user8@exemplo.com,4476.1,Belo Horizonte,2026/02/18-04:02:12
10,Pessoa 9,61,user9@exemplo.com,9242.02,null,2026/02/18-04:02:12


In [0]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, date_format, expr, coalesce

# -----------------------------
# 1 Inicializa Spark
# -----------------------------
spark = SparkSession.builder \
    .appName("Bronze_to_Silver") \
    .getOrCreate()

# -----------------------------
# 2 Lê a tabela Bronze
# -----------------------------
bronze_table = "saas_project.core.bronze_raw"
df = spark.table(bronze_table)

# -----------------------------
# 3 Normaliza nomes das colunas
# -----------------------------
def normalizar_coluna(nome):
    nome = nome.lower()
    nome = re.sub(r'[^a-z0-9_]', '_', nome)
    nome = re.sub(r'_+', '_', nome)
    nome = nome.strip('_')
    return nome

df = df.toDF(*[normalizar_coluna(c) for c in df.columns])

# -----------------------------
# 4 Remove duplicados e linhas totalmente nulas
# -----------------------------
df = df.dropDuplicates()
df = df.na.drop(how='all')

# -----------------------------
# 5 Limpa strings e transforma valores inválidos em NULL
# -----------------------------
valores_invalidos = ["NULL", "UNKNOWN", "ERROR", "NA", "N/A", ""]
for c, t in df.dtypes:
    if t == 'string':
        df = df.withColumn(c, trim(col(c)))
        df = df.withColumn(
            c,
            expr(f"CASE WHEN lower({c}) IN ({','.join([f'\'{v.lower()}\'' for v in valores_invalidos])}) THEN NULL ELSE {c} END")
        )

# -----------------------------
# 6 Trata TODAS as colunas do cliente que tenham data/hora juntas
# -----------------------------
# Nome indicando data/hora ou tipo timestamp
colunas_cliente = [
    c for c, t in df.dtypes
    if any(k in c.lower() for k in ["date", "data", "dia", "hora", "time"]) or t.startswith("timestamp")
]

formatos = [
    "yyyy-MM-dd HH:mm:ss",
    "dd/MM/yyyy HH:mm:ss",
    "MM/dd/yyyy HH:mm:ss",
    "yyyy-MM-dd",
    "dd/MM/yyyy",
    "MM/dd/yyyy"
]

for c in colunas_cliente:
    if c != "ingestion_time" and df.filter(col(c).isNotNull()).count() > 0:
        ts_col = c + "_ts"
        tipo_col = dict(df.dtypes)[c]

        # Só converte para timestamp se for string ou timestamp
        if tipo_col == 'string':
            df = df.withColumn(ts_col, expr(f"try_to_timestamp({c}, '{formatos[0]}')"))
            for f in formatos[1:]:
                df = df.withColumn(ts_col, coalesce(col(ts_col), expr(f"try_to_timestamp({c}, '{f}')")))
        elif tipo_col.startswith("timestamp"):
            df = df.withColumn(ts_col, col(c))
        else:
            # ignora colunas numéricas, não tenta date_format
            continue

        # Cria colunas separadas
        df = df.withColumn(c+"_date", date_format(col(ts_col), "dd-MM-yyyy"))
        df = df.withColumn(c+"_time", date_format(col(ts_col), "HH:mm:ss"))

        # Remove original + intermediário
        df = df.drop(c)
        df = df.drop(ts_col)

# -----------------------------
# 7 a Mantém ingestion_time como última coluna
# -----------------------------
if "ingestion_time" in df.columns:
    cols = [c for c in df.columns if c != "ingestion_time"] + ["ingestion_time"]
    df = df.select(*cols)

# -----------------------------
# 8 Mostra DataFrame final
# -----------------------------
df.show(5, truncate=False)
df.printSchema()

# -----------------------------
# 9 Salva no Silver Delta (Unity Catalog)
# -----------------------------
silver_table = "saas_project.core.silver_data"
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(silver_table)

print(" Dados processados e salvos no Silver profissional e robusto!")



+---+--------+-----+-----------------+-------+--------------+-------------------+
|id |nome    |idade|email            |salario|cidade        |ingestion_time     |
+---+--------+-----+-----------------+-------+--------------+-------------------+
|1  |Pessoa 0|26   |user0@exemplo.com|2036.32|NULL          |2026/02/18-04:02:12|
|2  |Pessoa 1|68   |user1@exemplo.com|3705.09|Rio           |2026/02/18-04:02:12|
|3  |Pessoa 2|67   |user2@exemplo.com|5693.16|São Paulo     |2026/02/18-04:02:12|
|4  |Pessoa 3|38   |user3@exemplo.com|9376.51|Belo Horizonte|2026/02/18-04:02:12|
|5  |Pessoa 4|53   |user4@exemplo.com|6392.52|NULL          |2026/02/18-04:02:12|
+---+--------+-----+-----------------+-------+--------------+-------------------+
only showing top 5 rows
root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- email: string (nullable = true)
 |-- salario: double (nullable = true)
 |-- cidade: string (nullable = true)
 |-- ingest

In [0]:
display(df.limit(10))

id,nome,idade,email,salario,cidade,ingestion_time
1,Pessoa 0,26,user0@exemplo.com,2036.32,null,2026/02/18-04:02:12
2,Pessoa 1,68,user1@exemplo.com,3705.09,Rio,2026/02/18-04:02:12
3,Pessoa 2,67,user2@exemplo.com,5693.16,São Paulo,2026/02/18-04:02:12
4,Pessoa 3,38,user3@exemplo.com,9376.51,Belo Horizonte,2026/02/18-04:02:12
5,Pessoa 4,53,user4@exemplo.com,6392.52,null,2026/02/18-04:02:12
6,Pessoa 5,49,user5@exemplo.com,7172.05,Rio,2026/02/18-04:02:12
7,Pessoa 6,null,user6@exemplo.com,3802.53,null,2026/02/18-04:02:12
8,Pessoa 7,47,user7@exemplo.com,4748.2,null,2026/02/18-04:02:12
9,Pessoa 8,27,user8@exemplo.com,4476.1,Belo Horizonte,2026/02/18-04:02:12
10,Pessoa 9,61,user9@exemplo.com,9242.02,null,2026/02/18-04:02:12
